Realizando Engenharia de Atributos e criando um modelo de arvore de decisão

Primeiro passo para começarmos a análise é importando as bibliotecas que iremos utilizar.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,  StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

Realizando a importação do arquivo com a extensão CSV e visualizando quantas linhas e colunas temos dentro do arquivo

In [2]:
dataset = pd.read_csv("credito4.csv", sep=";")
#visulizar
dataset.shape

(1000, 8)

Verificando as primeiras linhas do arquivo importado com o comando .head()

In [3]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


Separando as variáveis dependentes (X) da variavel independente (Y), a coluna CLASSE é a que queremos realizar a previsão


In [4]:
#separamos x de y
y = dataset['CLASSE']
X = dataset.iloc[:,0:7]

Realizando a verificação dos valores faltantes no dataframe.
Verificamos que as colunas SALDO_ATUAL e ESTADOSIVIL contém valores em branco.

In [5]:
#Tratamento de valores faltantes
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

Com a coluna SALDO_ATUAL vou preencher os valores faltantes com a mediana da coluna.
Então criei a variavel mediana para receber esse valor.

In [6]:
#preenche saldo atual pela mediana
mediana = X['SALDO_ATUAL'].median()
mediana

2323.0

Preenchendo os valores faltantes com a mediana e em seguida verificando se ainda contém algum valor faltando nesta coluna.

In [7]:
X['SALDO_ATUAL'].fillna(mediana, inplace=True)
#checa resultado
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

Na coluna ESTADOCIVIL iremos preencher com a média MODA

Verificando qual é o maior valor das categorias da coluna, e percebemos que masculino solteiro é a média MODA

In [8]:
#verifica qual a "moda" do estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

Substituimos os valores faltantes pela media moda (masculino solteiro) na coluna ESTADOCIVIL e em seguida verificamos se ainda há algum valor faltante nas colunas.

In [9]:
#substitui o estado civil pela moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
#checa resultado
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

Realizando tratamento dos OUTLIERS do dataframe na coluna SALDO_ATUAL

A variavel desv está com o valor do desvio padrão da coluna SALDO_ATUAL

In [10]:
#veficando qual é o valor do desvio padrão
desv = X['SALDO_ATUAL'].std()
desv

685936688.9820067

Definimos que será um OUTLIER quando o valor estiver 2 vezes acima do desvio padrão

In [11]:
#checamos se algum SALDO_ATUAL atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ,'SALDO_ATUAL'] 

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

Percemos que nas linhas 127 e 160 temos dois OUTLIERS que iremos substituir pela mediana

In [12]:
#calculando a mediana da coluna SALDO_ATUAL
mediana =X['SALDO_ATUAL'].median()
mediana

2323.0

Atribuindo a mediana onde o valor da coluna SALDO_ATUAL está 2 vezes acima do desvio padrão.

E verificamos se ainda há algum valor OUTLIER na coluna.

In [13]:
#atribuimos
X.loc[X['SALDO_ATUAL'] >=  2 * desv, 'SALDO_ATUAL'] = mediana
#checamos se algum atende critério
X.loc[X['SALDO_ATUAL'] >=  2 * desv ] 

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


Na coluna PROPOSITO, verificamos que temos muitas categorias, então realizamos um databinding para diminuir um pouco a quantidade.

In [14]:
#data binding
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

Na coluna PROPOSITO onde tem as categorias: Eletrodomésticos, mobilia/equipamento e educação, substituimos pela palavra: outros.

In [15]:
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'mobilia/equipamento', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'educação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

PROPOSITO
carro novo      234
carro usado     103
negócios         97
obras            22
outros          255
qualificação      9
radio/tv        280
dtype: int64

Tratamento da coluna DATA

Realizamos a extração das caracteristicas da data, primeiro transformamos a coluna para tipo datatime

In [16]:
#extração de caracteristicas da data
#transforma coluna em data
X['DATA'] = pd.to_datetime(X['DATA'], format="%d/%m/%Y")

In [17]:
X['DATA']

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

Criando 3 novas colunas (atributos), com as caracteristicas da coluna DATA

In [19]:
#criamos 3 novos atributos (colunas)
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [20]:
X['DIASEMANA']

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

In [21]:
X['MES']

0      1
1      1
2      1
3      1
4      1
      ..
995    6
996    6
997    7
998    7
999    7
Name: MES, Length: 1000, dtype: int32

In [22]:
X['ANO']

0      2019
1      2020
2      2020
3      2019
4      2018
       ... 
995    2018
996    2018
997    2018
998    2019
999    2018
Name: ANO, Length: 1000, dtype: int32

Realizando um label encoder nas colunas: ESTADOCIVIL, PROPOSITO e DIASEMANA

O Label Encoding é um método que atribui cada categoria a um número inteiro único. Ele é útil quando as categorias não têm qualquer ordem intrínseca.

Primeiramente verifico as categorias unicas de cada coluna

In [23]:
#label encoder estado civil, proposito e dia semana
X['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [24]:
X['PROPOSITO'].unique()

array(['radio/tv', 'outros', 'carro novo', 'carro usado', 'negócios',
       'obras', 'qualificação'], dtype=object)

In [25]:
X['DIASEMANA'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

Após a verificação criamos um objeto para ser do tipo LabelEncoder().

E usamos esse objeto nas 3 colunas para ser feito o tratamento com labelencoder1.fit_transform

In [26]:
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

Verificando como ficou o dataframe após o label encoder, nota-se que as colunas agora estão com o tipo numérico.

In [27]:
X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


Tratamento da coluna OUTROSPLANOSPGTO com o one hot encoding

O One-Hot Encoding é um método que transforma cada categoria em uma coluna binária (com valores 0 ou 1). Ele é útil quando as categorias têm uma ordem intrínseca

In [28]:
#one hot encoding
#OUTROSPLANOSPGTO
outros = X['OUTROSPLANOSPGTO'].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

Criamos um dataframe para pegar as colunas que serão criadas com o one hot encoding

In [29]:
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')

Visualizando o datafram z

In [30]:
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


Visualizando o dataframe X

In [31]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,4,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,6,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,6,2019,7,4


Realizando a padronização das colunas: SALDO_ATUAL, RESIDENCIADESDE e IDADE

A padronização tem o objetivo de transformar todas as variáveis na mesma ordem de grandeza.

Criamos um objeto para realizar a padronização que é o StandardScaler() e geramos um novo dataframe m para receber as colunas padronizadas.

In [32]:
#padronização Z-score
#gera objeto numpy
sc = StandardScaler()
m = sc.fit_transform( X.iloc[:,0:3])

Visualizando o dataframe m

In [33]:
m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

Após os tratamento das colunas, iremos realizar a concatenação dos dados dos dataframes X, z e m

In [34]:
#concatena os dados
X = pd.concat([X,z,pd.DataFrame(m, columns=['SALDO_ATUAL_N','RESIDENCIADESDE_N','IDADE_N'])], axis=1)

In [35]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,5,False,True,False,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,6,False,True,False,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4,False,True,False,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,6,False,True,False,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,False,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,4,2018,6,0,False,True,False,-0.543562,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2,False,True,False,0.211898,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,6,2018,7,5,False,True,False,-0.875522,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,6,2019,7,4,False,True,False,-0.504738,1.046987,-0.687363


Removendo as colunas que não serão usadas na criação do modelo.

In [36]:
#remover: saldo_atual e residenciadesde e idade que já foram normalizados
#outros planospagto foi feito onehot
#data foi extraida características
#OUTROS_banco por causa da dumy variable trap
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA', 'OUTROS_banco'], inplace=True)

In [37]:
X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,6,2019,1,5,True,False,-0.745516,1.046987,1.639276
1,0,6,2020,1,6,True,False,0.957740,-0.765977,-0.740241
2,3,4,2020,1,4,True,False,-0.415337,0.140505,0.687469
3,3,4,2019,1,6,True,False,1.645526,1.046987,0.475956
4,3,0,2018,1,6,True,False,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,4,2018,6,0,True,False,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,True,False,0.211898,1.046987,0.211566
997,3,6,2018,7,5,True,False,-0.875522,1.046987,0.105809
998,3,6,2019,7,4,True,False,-0.504738,1.046987,-0.687363


Criando o modelo de Arvore de decisão, sendo que 30% dos dados estão sendo separados para base de teste.

In [38]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y,test_size = 0.3,random_state = 0)
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)

Obtendo 68% de taxa de acerto do modelo de arvore de decisão.

In [39]:
taxa_acerto

0.68